## PURPOSE OF THE NOTEBOOK + TITLE OF THE PROJECT

In [ ]:
import requests
from requests.adapters import HTTPAdapter
import json

import pandas as pd
from datetime import date
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
import time
import numpy as np
import re

In [ ]:
links = ['https://www.ravelry.com/patterns/search#pc=medical&sort=name&view=captioned_thumbs&page=1','https://www.ravelry.com/patterns/search#pc=medical&sort=name&view=captioned_thumbs&page=2','https://www.ravelry.com/patterns/search#pc=medical&sort=name&view=captioned_thumbs&page=3','https://www.ravelry.com/patterns/search#pc=medical&sort=name&view=captioned_thumbs&page=4']

### Get the ID's of all the recipes in the medical knitting recipe category

In [ ]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

url = links[0]
await page.goto(url)

# Use playwright to login to the site. The xxxxx replace the username and password:
await page.locator("#user_login").fill('xxxxx')
await page.locator("#user_password").fill('xxxxx')
await page.locator("#login_form > div.splash_page__submit > div.splash_page__submit__login > button").click()

time.sleep(5)

pattern_ids = []

# Scrape each of the four pages:
for link in links:
    await page.goto(link)
    time.sleep(5)
    html = await page.content()
    doc = BeautifulSoup(html)

    content_card = doc.find_all(class_= "pattern captioned_pattern captioned_search_result")

    pattern_id = []

    # Loop through each pattern on the page and get the information:
    for card in content_card:
        pattern = {}
        pattern['name'] = card.find('a').get('title')
        pattern['url'] = card.find('a').get('href')

        find_id = card.get('id')
        pattern['id'] = re.findall(r'\d+', find_id)[0]

        # And append the pattern data to the list:
        pattern_id.append(pattern)
    pattern_ids.append(pattern_id)

### And take the list of lists (with some dictionaries inside) created in the scrape and turn it into one flat list.

This allows me to turn it into a dataframe and then into a csv file.

In [ ]:
flat_list = []
for ids in pattern_ids:
    for id in ids:
        flat_list.append(id)

id_df = pd.DataFrame(flat_list)

In [ ]:
id_df.to_csv("data/patterns.csv", index=False)

## Get information on the patterns

In [ ]:
df = pd.read_csv("data/patterns.csv")

In [ ]:
url_list = df.url.values.tolist()

In [ ]:
project_numbers = []

for url in url_list:
    await page.goto(url)

    time.sleep(0.5)

    project = {}

    html = await page.content()
    doc = BeautifulSoup(html)

    type = doc.find_all(class_='value')
    type = type[1].text
    type = re.sub(r'\n', '', type)

    project['craft_type'] = type
    
    projects = doc.find(class_= "pattern_people summary_box_people").text
    # replace \n\n\n with nothing
    projects = re.sub(r'\n\n\n', '', projects)
    project['project_numbers'] = re.sub(r' projec\D*,\nin \d* queues\n\n', '', projects)
    
    project_numbers.append(project)
await browser.close()

In [ ]:
df = pd.DataFrame(project_numbers)
df.to_csv("data/patterns_info.csv", index=False)

In [ ]:
patterns = pd.read_csv("data/patterns.csv")
patterns_info = pd.read_csv("data/patterns_info.csv")

In [ ]:
# Merge the two dataframes using concat
total = pd.concat([patterns, patterns_info], axis=1)

# Give the recipe with no projects the value 0
total = total.replace(regex=r'no projects\n', value=0)

In [ ]:
# And save it as a csv
total.to_csv("data/patterns_total.csv", index=False)